In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
from PIL import Image
import requests
import torch
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
# 모델 load

feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b3-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b3-finetuned-ade-512-512")

/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [5]:
def segment_image(input_dir, output_dir):
  dir_path = input_dir # 이미지 있는 폴더 경로
  file_lst = os.listdir(dir_path) # 폴더 내 모든 파일 list
  file_lst_jpg = []

  for i in file_lst: # 리스트에 모든 이미지 경로 추가
    file_split = i.split('.')
    if file_split[-1] == 'jpg':
      file_lst_jpg.append(dir_path+'/'+i)

  class_colors = np.full((183,3), 0, dtype=np.uint16)
  class_colors[1] = [255,0,0] # 도로
  class_colors[2] = [0,255,0] # 잔디
  class_colors[3] = [0,0,255] # 나무

  for i in file_lst_jpg:
    image = Image.open(i)
    original_width, original_height = image.size
    original_size = (original_height,original_width)
    inputs = feature_extractor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
    probabilities = torch.softmax(logits, dim=1)
    predicted_labels = torch.argmax(probabilities, dim=1)
    upsampled_labels = torch.nn.functional.interpolate(predicted_labels.float().unsqueeze(0), size=original_size, mode="nearest").squeeze(0).long() # 사진크기로 리사이징

    upsampled_labels[upsampled_labels==0] = 9 # wall -> grass
    upsampled_labels[upsampled_labels==13] = 9 # earth, ground -> grass
    upsampled_labels[upsampled_labels==52] = 6 # path -> road
    upsampled_labels[upsampled_labels==5] = 4 # ceiling -> tree

    upsampled_labels[upsampled_labels==6] = 1 # 도로
    upsampled_labels[upsampled_labels==9] = 2 # 잔디
    upsampled_labels[upsampled_labels==4] = 3 # 나무

    upsampled_labels = upsampled_labels.numpy()
    upsampled_labels[np.isin(upsampled_labels, [1,2,3], invert=True)] = 0

    filename = i.split('/')[-1][:-4]
    np.savetxt(output_dir+'/'+filename+'.txt', upsampled_labels[0], fmt='%d')
    # np.save(output_dir+'/'+filename+'.txt', upsampled_labels[0])

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_42_58_F/', '/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_42_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_37_58_F/', '/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_37_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_38_58_F/', '/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_38_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_39_58_F/', '/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_39_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_40_58_F/', '/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_40_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_41_58_F/', '/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_41_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_43_58_F/', '/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_43_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_44_58_F/', '/content/drive/MyDrive/골프영상/주행영상_5/REC_2022_10_05_11_44_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_36_58_F/', '/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_36_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_35_58_F/', '/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_35_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_34_58_F/', '/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_34_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_33_58_F/', '/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_33_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_32_58_F/', '/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_32_58_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_31_57_F/', '/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_31_57_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_30_57_F/', '/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_30_57_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_29_57_F/', '/content/drive/MyDrive/골프영상/주행영상_6/REC_2022_10_05_11_29_57_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_28_57_F/', '/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_28_57_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_27_57_F/', '/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_27_57_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_26_57_F/', '/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_26_57_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_25_57_F/', '/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_25_57_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_24_57_F/', '/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_24_57_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_23_57_F/', '/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_23_57_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_22_28_F/', '/content/drive/MyDrive/골프영상/주행영상_7/REC_2022_10_05_11_22_28_F/결과')

In [6]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_12_34_01_F/', '/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_12_34_01_F/결과')

In [7]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_12_33_01_F/', '/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_12_33_01_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_11_21_28_F/', '/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_11_21_28_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_11_20_28_F/', '/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_11_20_28_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_11_19_28_F/', '/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_11_19_28_F/결과')

In [ ]:
segment_image('/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_11_18_29_F/', '/content/drive/MyDrive/골프영상/주행영상_8/REC_2022_10_05_11_18_29_F/결과')

In [ ]:
import random

In [ ]:
img_dir = '/content/drive/MyDrive/계명대학교_최현성_분석연습/골프영상/주행영상_9/REC_2022_10_05_12_30_01_F'

In [ ]:
txt_dir = '/content/drive/MyDrive/계명대학교_최현성_분석연습/이미지짜르기/테스트_5'

In [ ]:
txt_lst = os.listdir(txt_dir)
file_title = random.choice(txt_lst)[:-4]


In [ ]:
sample_txt = txt_dir + '/' + file_title + '.txt'
sample_img = img_dir + '/' + file_title + '.jpg'

In [ ]:
segment_text = np.loadtxt(sample_txt) # np.loadtxt로 txt파일 load
image = Image.open(sample_img) # image.open으로 image파일 open

In [ ]:
segment_value = torch.Tensor(segment_text)
segment_value = segment_value.int()

In [ ]:
class_colors = np.full((183,3), 0, dtype=np.uint16)
class_colors[1] = [255,0,0]
class_colors[2] = [0,255,0]
class_colors[3] = [0,0,255]

# 클래스별 색상설정

In [ ]:
colored_masks = class_colors[segment_value]
image_np = np.array(image)

In [ ]:
alpha = 0.5  # 원하는 투명도 값 (0.0 ~ 1.0)
overlay_np = (colored_masks * alpha).astype(np.uint8)

In [ ]:
combined_np = (np.array(image) * (1 - alpha) + overlay_np).astype(np.uint8)

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(colored_masks)
plt.title("Colored Mask")
plt.axis('off')
plt.show()